# An Argo float


This tutorial shows how simple it is to construct a Kernel in Parcels that mimics the [vertical movement of Argo floats](https://www.aoml.noaa.gov/phod/argo/images/argo_float_mission.jpg).

We first define the kernels for each phase of the Argo cycle.

In [ ]:
import numpy as np

# Define the new Kernels that mimic Argo vertical movement
driftdepth = 1000  # maximum depth in m
maxdepth = 2000  # maximum depth in m
vertical_speed = 0.10  # sink and rise speed in m/s
cycletime = (
    10 * 86400
)  # total time of cycle in seconds  # TODO update to "timedelta64[s]"
drifttime = 9 * 86400  # time of deep drift in seconds


def ArgoPhase1(particles, fieldset):
    dt = particles.dt / np.timedelta64(1, "s")  # convert dt to seconds

    def SinkingPhase(p):
        """Phase 0: Sinking with vertical_speed until depth is driftdepth"""
        p.ddepth += vertical_speed * dt
        p.cycle_phase = np.where(p.depth + p.ddepth >= driftdepth, 1, p.cycle_phase)
        p.ddepth = np.where(
            p.depth + p.ddepth >= driftdepth, driftdepth - p.depth, p.ddepth
        )

    SinkingPhase(particles[particles.cycle_phase == 0])


def ArgoPhase2(particles, fieldset):
    dt = particles.dt / np.timedelta64(1, "s")  # convert dt to seconds

    def DriftingPhase(p):
        """Phase 1: Drifting at depth for drifttime seconds"""
        p.drift_age += dt
        p.cycle_phase = np.where(p.drift_age >= drifttime, 2, p.cycle_phase)
        p.drift_age = np.where(p.drift_age >= drifttime, 0, p.drift_age)

    DriftingPhase(particles[particles.cycle_phase == 1])


def ArgoPhase3(particles, fieldset):
    dt = particles.dt / np.timedelta64(1, "s")  # convert dt to seconds

    def SecondSinkingPhase(p):
        """Phase 2: Sinking further to maxdepth"""
        p.ddepth += vertical_speed * dt
        p.cycle_phase = np.where(p.depth + p.ddepth >= maxdepth, 3, p.cycle_phase)
        p.ddepth = np.where(
            p.depth + p.ddepth >= maxdepth, maxdepth - p.depth, p.ddepth
        )

    SecondSinkingPhase(particles[particles.cycle_phase == 2])


def ArgoPhase4(particles, fieldset):
    dt = particles.dt / np.timedelta64(1, "s")  # convert dt to seconds

    def RisingPhase(p):
        """Phase 3: Rising with vertical_speed until at surface"""
        p.ddepth -= vertical_speed * dt
        p.temp = fieldset.thetao[p.time, p.depth, p.lat, p.lon]
        p.cycle_phase = np.where(
            p.depth + p.ddepth <= fieldset.mindepth, 4, p.cycle_phase
        )
        p.ddepth = np.where(
            p.depth + p.ddepth <= fieldset.mindepth,
            fieldset.mindepth - p.depth,
            p.ddepth,
        )

    RisingPhase(particles[particles.cycle_phase == 3])


def ArgoPhase5(particles, fieldset):
    def TransmittingPhase(p):
        """Phase 4: Transmitting at surface until cycletime is reached"""
        p.cycle_phase = np.where(p.cycle_age >= cycletime, 0, p.cycle_phase)
        p.cycle_age = np.where(p.cycle_age >= cycletime, 0, p.cycle_age)
        p.temp = np.nan  # no temperature measurement when at surface

    TransmittingPhase(particles[particles.cycle_phase == 4])


def ArgoPhase6(particles, fieldset):
    dt = particles.dt / np.timedelta64(1, "s")  # convert dt to seconds
    particles.cycle_age += dt  # update cycle_age

And then we can run Parcels with this 'custom kernel'.

Below we use the horizontal velocity fields of CopernicusMarine, which are provided as example_data with Parcels.


In [ ]:
from datetime import timedelta

import xarray as xr

import parcels

# Load the CopernicusMarine data in the Agulhas region from the example_datasets
example_dataset_folder = parcels.download_example_dataset(
    "CopernicusMarine_data_for_Argo_tutorial"
)

ds = xr.open_mfdataset(f"{example_dataset_folder}/*.nc", combine="by_coords")

# TODO check how we can get good performance without loading full dataset in memory
ds.load()  # load the dataset into memory

fieldset = parcels.FieldSet.from_copernicusmarine(ds)
fieldset.add_constant("mindepth", 1.0)

# Define a new Particle type including extra Variables
ArgoParticle = parcels.Particle.add_variable(
    [
        parcels.Variable("cycle_phase", dtype=np.int32, initial=0.0),
        parcels.Variable(
            "cycle_age", dtype=np.float32, initial=0.0
        ),  # TODO update to "timedelta64[s]"
        parcels.Variable("drift_age", dtype=np.float32, initial=0.0),
        parcels.Variable("temp", dtype=np.float32, initial=np.nan),
    ]
)

# Initiate one Argo float in the Agulhas Current
pset = parcels.ParticleSet(
    fieldset=fieldset,
    pclass=ArgoParticle,
    lon=[32],
    lat=[-31],
    depth=[fieldset.mindepth],
)

# combine Argo vertical movement kernel with built-in Advection kernel
kernels = [
    ArgoPhase1,
    ArgoPhase2,
    ArgoPhase3,
    ArgoPhase4,
    ArgoPhase5,
    ArgoPhase6,
    parcels.AdvectionRK4,
]

# Create a ParticleFile object to store the output
output_file = parcels.ParticleFile(
    store="argo_float.zarr",
    outputdt=timedelta(minutes=15),
    chunks=(1, 500),  # setting to write in chunks of 500 observations
)

# Now execute the kernels for 30 days, saving data every 30 minutes
pset.execute(
    kernels,
    runtime=timedelta(days=30),
    dt=timedelta(minutes=15),
    output_file=output_file,
)

Now we can plot the trajectory of the Argo float with some simple calls to netCDF4 and matplotlib.

First plot the depth as a function of time, with the temperature as color (only on the upcast).

In [ ]:
ds_out = xr.open_zarr(
    output_file.store, decode_times=False
)  # TODO fix without using decode_times=False
x = ds_out["lon"][:].squeeze()
y = ds_out["lat"][:].squeeze()
z = ds_out["z"][:].squeeze()
time = ds_out["time"][:].squeeze() / 86400  # convert time to days
temp = ds_out["temp"][:].squeeze()

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(13, 6))
ax = plt.axes()
ax.plot(time, z, color="gray")
cb = ax.scatter(time, z, c=temp, s=20, marker="o", zorder=2)
ax.set_xlabel("Time [days]")
ax.set_ylabel("Depth (m)")
ax.invert_yaxis()
fig.colorbar(cb, label="Temperature (°C)")
plt.show()

We can also make a 3D plot of the trajectory colored by temperature.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(13, 8))
ax = plt.axes(projection="3d")
ax.plot3D(x, y, z, color="gray")
cb = ax.scatter(x, y, z, c=temp, s=20, marker="o", zorder=2)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_zlabel("Depth (m)")
ax.set_zlim(np.max(z), 0)
fig.colorbar(cb, label="Temperature (°C)")
plt.show()